In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy pandas qiskit-ibm-transpiler

*Gschätzte QPU-Nutzung: Kei (HIIWIS: Das Tutorial füert kei Jobs uus, will's sich uf Transpilation konzentriert)*

## Hintergrund
De **Qiskit KI-gstützt Transpiler-Service (QTS)** füert maschinells-Lerne-basierti Optimierige sowohl i Routing- als au i Synthese-Passes ii. Dää KI-Modi sind entwicklet worde, zum d'Iischränkige vo de traditionelle Transpilation aagaa, bsunders für grossi Schaltige und komplexe Hardware-Topologie.

Ab **Juli 2025** isch de **Transpiler-Service** zur neue IBM Quantum&reg; Plattform migriert worde und isch nümme verfüegbar. Für die neueschte Updates zum Status vom Transpiler-Service lueg id [Transpiler-Service-Dokumentation](/guides/qiskit-transpiler-service). Du chasch de KI-Transpiler wiiterhin lokal verwende, ähnlich wie bi de Standard-Qiskit-Transpilation. Ersetz eifach `generate_preset_pass_manager()` dur `generate_ai_pass_manager()`. Die Funktion konstruiert en Pass-Manager, wo d'KI-gstützte Routing- und Synthese-Passes direkt i dis lokale Transpilations-Workflow integriert.

### Hauptmerkmale vo de KI-Passes
- Routing-Passes: KI-gstützts Routing chan Qubit-Pfäd dynamisch basierend uf de spezifischi Schaltig und s'Backend aapasse und de Bedarf a übermässige SWAP-Gates reduziere.
    - `AIRouting`: Layout-Uuswahl und Schaltig-Routing

- Synthese-Passes: KI-Technike optimiere d'Zerlëgig vo Multi-Qubit-Gates und minimiere d'Aazahl vo de Zwei-Qubit-Gates, wo typischerwiis afälliger für Fähler sind.
    - `AICliffordSynthesis`: Clifford-Gate-Synthese
    - `AILinearFunctionSynthesis`: Synthese vo Linear-Funktionsschaltige
    - `AIPermutationSynthesis`: Synthese vo Permutationsschaltige
    - `AIPauliNetworkSynthesis`: Synthese vo Pauli-Netzwerkschaltige (nume im Qiskit Transpiler Service verfüegbar, nöd i de lokale Umgëbig)

- Vergliich mit traditioneller Transpilation: De Standard-Qiskit-Transpiler isch es robustes Wärkzüüg, wo es breits Spektrum vo Quanteschaltige effektiv hanthabe chan. Wenn Schaltige aber grösser wärde oder Hardware-Konfiguratione komplexer wärde, chöi KI-Passes zuesätzlichi Optimiergsgewinne liefere. Dur d'Verwendig vo glehrte Modell für Routing und Synthese verfeineret QTS Schaltigslayouts wiiter und reduziert de Overhead für useforderendi oder gross aaleiti Quanteufgabe.

Das Tutorial evaluiert d'KI-Modi under Verwendig vo Routing- und ou vo Synthese-Passes und verglicht d'Ergebnis mit traditioneller Transpilation, zum hervörhebe, wo KI Leistigsvordel bietet.

Für wiitri Details zu de verfüegbare KI-Passes lueg id [KI-Passes-Dokumentation](/guides/ai-transpiler-passes).

### Worum KI für Quanteschaltig-Transpilation verwände?
Wil Quanteschaltige i Grössi und Komplexität zuenemme, hend traditionelli Transpilationsmethode Schwiirigkeite, Layouts z'optimiere und Gate-Aazahle effizient z'reduziere. Grösseri Schaltige, bsunders die mit Hunderte vo Qubits, stelle erhebliche Usefordrige a s'Routing und d'Synthese dar, wëge Gerätebschränkige, begränzter Konnektivität und Qubit-Fählerrate.

Daa bietet d'KI-gstützt Transpilation e potenzielli Lösig. Dur d'Nutzig vo maschinelle Lehrntechnike chan de KI-gstützt Transpiler i Qiskit gschider Entscheidige über Qubit-Routing und Gate-Synthese träffe, was zu besserer Optimierig vo gross aaleite Quanteschaltige füert.

### Churzi Benchmarking-Ergebnis
![Graph showing AI transpiler performance against Qiskit](../docs/images/tutorials/ai-transpiler-introduction/ai-transpiler-benchmarks.avif)

I Benchmarking-Tests het de KI-Transpiler konsistent flacheri Schaltige höcherer Qualität im Vergliich zum Standard-Qiskit-Transpiler produziert. Für dää Tests hend mir d'Standard-Pass-Manager-Strategie vo Qiskit verwändet, konfiguriert mit [`generate_preset_passmanager`]. Während die Standardstrategie oft effektiv isch, chan sie bi grössere oder komplexere Schaltige Schwiirigkeite haa. Im Gëgesatz dezue hend KI-gstützti Passes e durchschnittlichi Reduzierig vo de Zwei-Qubit-Gate-Aazahl um 24% und e Reduzierig vo de Schaltigstieffi um 36% für grossi Schaltige (100+ Qubits) bi de Transpilation uf d'Heavy-Hex-Topologie vo IBM Quantum Hardware erreicht. Wiitri Informatione zu däne Benchmarks findsch im [Blog.](https://www.ibm.com/quantum/blog/qiskit-performance)

Das Tutorial underschuecht d'wichtigste Vordel vo de KI-Passes und wie sie sich mit traditionelle Methode vergliche.

In [1]:
# This cell is hidden from users;
# it just disables a linting rule.
# ruff: noqa: F811

## Aaforderige

Stell vor em Aafang vo däm Tutorial sicher, dass du Folgendes installiert hesch:

* Qiskit SDK v1.0 oder höcher, mit Understützig für [Visualisierig](https://docs.quantum.ibm.com/api/qiskit/visualization)
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22 oder höcher
* Qiskit IBM&reg; Transpiler mit KI-Lokalmodus(`pip install 'qiskit-ibm-transpiler[ai-local-mode]'`)

## Setup

In [2]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import efficient_su2, PermutationGate
from qiskit.synthesis.qft import synth_qft_full
from qiskit.circuit.random import random_circuit, random_clifford_circuit
from qiskit.transpiler import generate_preset_pass_manager, CouplingMap
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_transpiler import generate_ai_pass_manager
from qiskit.synthesis.permutation import (
    synth_permutation_depth_lnn_kms,
    synth_permutation_basic,
)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import logging

seed = 42


# Used for generating permutation circuits in part two for comparison
def generate_permutation_circuit(width, pattern):
    circuit = QuantumCircuit(width)
    circuit.append(
        PermutationGate(pattern=pattern),
        qargs=range(width),
    )
    return circuit


# Creates a Bernstein-Vazirani circuit given the number of qubits
def create_bv_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits, num_qubits - 1)
    qc.x(num_qubits - 1)
    qc.h(qc.qubits)
    for i in range(num_qubits - 1):
        qc.cx(i, num_qubits - 1)
    qc.h(qc.qubits[:-1])
    return qc


# Transpile a circuit with a given pass manager and return metrics
def transpile_with_metrics(pass_manager, circuit):
    start = time.time()
    qc_out = pass_manager.run(circuit)
    elapsed = time.time() - start

    depth_2q = qc_out.depth(lambda x: x.operation.num_qubits == 2)
    gate_count = qc_out.size()

    return qc_out, {
        "depth_2q": depth_2q,
        "gate_count": gate_count,
        "time_s": elapsed,
    }


# Used for collecting metrics for part 3 of synthesis methods
def synth_transpile_with_metrics(qc, pm, pattern_id, method):
    start = time.time()
    qc = pm.run(qc)
    elapsed = time.time() - start

    return {
        "Pattern": pattern_id,
        "Method": method,
        "Depth (2Q)": qc.depth(lambda x: x.operation.num_qubits == 2),
        "Gates": qc.size(),
        "Time (s)": elapsed,
    }


# Ignore logs like "INFO:qiskit_ibm_transpiler.wrappers.ai_local_synthesis:Running Linear Functions AI synthesis on local mode"

logging.getLogger(
    "qiskit_ibm_transpiler.wrappers.ai_local_synthesis"
).setLevel(logging.WARNING)

# Teil I. Qiskit-Muster

Jetzt luege mir üs aa, wie mer de KI-Transpiler-Service mit ere einfache Quanteschaltig under Verwendig vo Qiskit-Muster verwände. De Schlüssel isch d'Erstellig vo eme `PassManager` mit `generate_ai_pass_manager()` anstatt vom Standard `generate_preset_pass_manager()`.

## Schritt 1: Klassischi Iigabe uf es Quanteproblem abilde

I däm Abschnitt teste mir de KI-Transpiler uf de `efficient_su2`-Schaltig, eme wiit verbreitete hardwareeffiziente Aasatz. Dää Schaltig isch bsunders relevant für variationelli Quantealgorithme (zum Biischpil VQE) und Quantum-Machine-Learning-Ufgabe, was sie zu eme ideale Testfall für d'Bewertig vo de Transpilationslöschtig macht.

D'`efficient_su2`-Schaltig beschtoot uus abwëchslende Schichte vo Ein-Qubit-Rotatione und verschränkende Gates wie CNOTs. Dää Schichte ermögliche e flexibli Erkundig vom Quantezuestandsruum, während d'Gate-Tieffi überschaubar blibt. Dur Optimierig vo dere Schaltig wänd mir d'Gate-Aazahl reduziere, d'Fidelität verbessere und Ruusche minimiere. Das macht sie zu eme starke Kandidat für s'Teschte vo de Effiziänz vom KI-Transpiler.

In [3]:
# For our transpilation, we will use a large circuit of 101 qubits
qc = efficient_su2(90, entanglement="circular", reps=1).decompose()

# Draw a smaller version of the circuit to get a visual representation
qc_small = efficient_su2(5, entanglement="circular", reps=1).decompose()
qc_small.draw(output="mpl")

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/c6e9c2c0-e02c-4276-bae8-d5692e60b6b8-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/c6e9c2c0-e02c-4276-bae8-d5692e60b6b8-0.avif)

## Schritt 2: Problem für d'Usführig uf Quantehardware optimiere
### Es Backend uuswähle
Für das Biischpil wähle mir s'am wenigschte usglaschtet betriebsbereit IBM Quantum Backend, wo kei Simulator isch und mindëschtens 100 Qubits het:

**Hiiwis:** Wil sich s'am wenigschte usglaschtet Backend im Lauf vo de Ziit chönti ändere, chöi für verschideni Durchläuf underschidlichi Gerät uusgwählt wärde. Gerät-spezifischi Eigeschafte, wie Coupling-Maps, chöi zu Underschid i de transpilierte Schaltige füere.

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=100
)
cm = backend.coupling_map
print(f"Using backend: {backend.name}")

Using backend: ibm_torino


### Create AI and traditional pass managers
To evaluate the effectiveness of the AI transpiler, we will perform two transpilation runs. First, we will transpile the circuit using the AI transpiler. Then, we will run a comparison by transpiling the same circuit without the AI transpiler, using traditional methods. Both transpilation processes will use the same coupling map from the chosen backend and the optimization level set to 3 for a fair comparison.

Both of these methods reflect the standard approach to create `PassManager` instances to transpile circuits in Qiskit.

In [5]:
pm_ai = generate_ai_pass_manager(
    optimization_level=3,
    ai_optimization_level=3,
    coupling_map=cm,
    include_ai_synthesis=True,  # used for part 3 when comparing synthesis methods
)

pm_no_ai = generate_preset_pass_manager(
    optimization_level=3,
    coupling_map=cm,
    seed_transpiler=seed,  # note that the AI pass manager does not currently support seeding
)

### KI- und traditionelli Pass-Manager ersch stelle
Zum d'Effektivität vom KI-Transpiler z'bewärte, füere mir zwei Transpilationslööf dur. Zersch transpiliere mir d'Schaltig mit em KI-Transpiler. Denn füere mir e Vergliich dur, indem mir dëselbi Schaltig ohni de KI-Transpiler mit traditionelle Methode transpiliere. Beidi Transpilationsprozess verwände dëselbi Coupling-Map vom gwählte Backend und s'Optimierigslevel wird uf 3 gsetzt, für e faire Vergliich.

Beidi Methode spiegele de Standardaasatz zur Erstellig vo `PassManager`-Instanze zur Transpilation vo Schaltige i Qiskit wider.

In [6]:
# Transpile using standard (non-AI) pass manager
_, metrics_no_ai = transpile_with_metrics(pm_no_ai, qc)
print(
    f"Standard transpilation: Depth (2q) {metrics_no_ai['depth_2q']}, "
    f"Gate count {metrics_no_ai['gate_count']}, Time {metrics_no_ai['time_s']}"
)

# Transpile using AI pass manager
_, metrics_ai = transpile_with_metrics(pm_ai, qc)
print(
    f"AI transpilation      : Depth (2q) {metrics_ai['depth_2q']}, "
    f"Gate count {metrics_ai['gate_count']}, Time {metrics_ai['time_s']}"
)

Standard transpilation: Depth (2q) 95, Gate count 458, Time 0.04650712013244629
AI transpilation      : Depth (2q) 90, Gate count 456, Time 0.9342479705810547


Transpilier d'Schaltige und zeichne d'Ziite uf.

In [7]:
# Circuits to benchmark
seed = 42
circuits = [
    {
        "name": "Random",
        "qc": random_circuit(num_qubits=30, depth=10, seed=seed),
    },
    {
        "name": "Clifford",
        "qc": random_clifford_circuit(
            num_qubits=40, num_gates=200, seed=seed
        ),
    },
    {
        "name": "QFT",
        "qc": synth_qft_full(num_qubits=20, do_swaps=False).decompose(),
    },
    {
        "name": "BV",
        "qc": create_bv_circuit(40),
    },
]

results = []

# Run the transpilation for each circuit and store the results
for circuit in circuits:
    qc_no_ai, metrics_no_ai = transpile_with_metrics(pm_no_ai, circuit["qc"])
    qc_ai, metrics_ai = transpile_with_metrics(pm_ai, circuit["qc"])

    print("Completed transpilation for", circuit["name"])

    results.append(
        {
            "Circuit": circuit["name"],
            "Depth 2Q (No AI)": metrics_no_ai["depth_2q"],
            "Gate Count (No AI)": metrics_no_ai["gate_count"],
            "Time (No AI)": metrics_no_ai["time_s"],
            "Depth 2Q (AI)": metrics_ai["depth_2q"],
            "Gate Count (AI)": metrics_ai["gate_count"],
            "Time (AI)": metrics_ai["time_s"],
        }
    )

df = pd.DataFrame(results)
df

Completed transpilation for Random
Completed transpilation for Clifford
Completed transpilation for QFT
Completed transpilation for BV


,Circuit,Depth 2Q (No AI),Gate Count (No AI),Time (No AI),Depth 2Q (AI),Gate Count (AI),Time (AI)
0,Random,37,221,0.039347,24,181,0.773718
1,Clifford,36,232,0.036633,43,267,1.097431
2,QFT,165,924,0.077458,130,913,3.660771
3,BV,65,155,0.024993,70,155,0.345522


Average percentage reduction for each metric. Positive are improvements, negative are degradations.

In [8]:
# Average reduction from non-AI to AI transpilation as a percentage
avg_reduction_depth = (
    (df["Depth 2Q (No AI)"] - df["Depth 2Q (AI)"]).mean()
    / df["Depth 2Q (No AI)"].mean()
    * 100
)
avg_reduction_gates = (
    (df["Gate Count (No AI)"] - df["Gate Count (AI)"]).mean()
    / df["Gate Count (No AI)"].mean()
    * 100
)
avg_reduction_time = (
    (df["Time (No AI)"] - df["Time (AI)"]).mean()
    / df["Time (No AI)"].mean()
    * 100
)

print(f"Average reduction in depth: {avg_reduction_depth:.2f}%")
print(f"Average reduction in gate count: {avg_reduction_gates:.2f}%")
print(f"Average reduction in transpilation time: {avg_reduction_time:.2f}%")

Average reduction in depth: 11.88%
Average reduction in gate count: 1.04%
Average reduction in transpilation time: -3193.95%


In [9]:
fig, axs = plt.subplots(1, 3, figsize=(21, 6))
df.plot(
    x="Circuit",
    y=["Depth 2Q (No AI)", "Depth 2Q (AI)"],
    kind="bar",
    ax=axs[0],
)
axs[0].set_title("Circuit Depth Comparison")
axs[0].set_ylabel("Depth")
axs[0].set_xlabel("Circuit")
axs[0].tick_params(axis="x", rotation=45)
df.plot(
    x="Circuit",
    y=["Gate Count (No AI)", "Gate Count (AI)"],
    kind="bar",
    ax=axs[1],
)
axs[1].set_title("Gate Count Comparison")
axs[1].set_ylabel("Gate Count")
axs[1].set_xlabel("Circuit")
axs[1].tick_params(axis="x", rotation=45)
df.plot(x="Circuit", y=["Time (No AI)", "Time (AI)"], kind="bar", ax=axs[2])
axs[2].set_title("Time Comparison")
axs[2].set_ylabel("Time (seconds)")
axs[2].set_xlabel("Circuit")
axs[2].tick_params(axis="x", rotation=45)
fig.suptitle(
    "Benchmarking AI transpilation vs Non-AI transpilation for various circuits"
)

plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/79b8d5d9-0f9d-42ca-9583-8bec17430014-0.avif" alt="Output of the previous code cell" />

The AI transpiler's performance varies significantly based on the type of circuit being optimized. In some cases, it achieves notable reductions in circuit depth and gate count compared to the standard transpiler. However, these improvements often come with a substantial increase in runtime.

For certain types of circuits, the AI transpiler may yield slightly better results in terms of circuit depth but may also lead to an increase in gate count and a significant runtime penalty. These observations suggest that the AI transpiler's benefits are not uniform across all circuit types. Instead, its effectiveness depends on the specific characteristics of the circuit, making it more suitable for some use cases than others.

## When should users choose AI-powered transpilation?

The AI-powered transpiler in Qiskit excels in scenarios where traditional transpilation methods struggle, particularly with large-scale and complex quantum circuits. For circuits involving hundreds of qubits or those targeting hardware with intricate coupling maps, the AI transpiler offers superior optimization in terms of circuit depth, gate count, and runtime efficiency. In benchmarking tests, it has consistently outperformed traditional methods, delivering significantly shallower circuits and reducing gate counts, which are critical for enhancing performance and mitigating noise on real quantum hardware.

Users should consider AI-powered transpilation when working with:
- Large circuits where traditional methods fail to efficiently handle the scale.
- Complex hardware topologies where device connectivity and routing challenges arise.
- Performance-sensitive applications where reducing circuit depth and improving fidelity are paramount.

# Part III. Explore AI-powered permutation network synthesis

Permutation networks are foundational in quantum computing, particularly for systems constrained by restricted topologies. These networks facilitate long-range interactions by dynamically swapping qubits to mimic all-to-all connectivity on hardware with limited connectivity. Such transformations are essential for implementing complex quantum algorithms on near-term devices, where interactions often span beyond nearest neighbors.

In this section, we highlight the synthesis of permutation networks as a compelling use case for the AI-powered transpiler in Qiskit. Specifically, the `AIPermutationSynthesis` pass leverages AI-driven optimization to generate efficient circuits for qubit permutation tasks. By contrast, generic synthesis approaches often struggle to balance gate count and circuit depth, especially in scenarios with dense qubit interactions or when attempting to achieve full connectivity.

We will walk through a Qiskit patterns example showcasing the synthesis of a permutation network to achieve all-to-all connectivity for a set of qubits. We will compare the performance of `AIPermutationSynthesis` against the standard synthesis methods in Qiskit. This example will demonstrate how the AI transpiler optimizes for lower circuit depth and gate count, highlighting its advantages in practical quantum workflows. To activate the AI synthesis pass, we will use the `generate_ai_pass_manager()` function with the `include_ai_synthesis` parameter set to `True`.

## Step 1: Map classical inputs to a quantum problem

To represent a classical permutation problem on a quantum computer, we start by defining the structure of the quantum circuits. For this example:

1. Quantum circuit initialization:
   We allocate 27 qubits to match the backend we will use, which has 27 qubits.

2. Apply permutations:
   We generate ten random permutation patterns (`pattern_1` through `pattern_10`) using a fixed seed for reproducibility. Each permutation pattern is applied to a separate quantum circuit (`qc_1` through `qc_10`).

3. Circuit decomposition:
   Each permutation operation is decomposed into native gate sets compatible with the target quantum hardware. We analyze the depth and the number of two-qubit gates (nonlocal gates) for each decomposed circuit.

The results provide insight into the complexity of representing classical permutation problems on a quantum device, demonstrating the resource requirements for different permutation patterns.

In [10]:
# Parameters
width = 27
num_circuits = 10

# Set random seed
np.random.seed(seed)


# Generate random patterns and circuits
patterns = [
    np.random.permutation(width).tolist() for _ in range(num_circuits)
]
circuits = {
    f"qc_{i}": generate_permutation_circuit(width, pattern)
    for i, pattern in enumerate(patterns, start=1)
}

# Display one of the circuits
circuits["qc_1"].decompose(reps=3).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/76a3e847-0808-4413-bd0c-c760cd2df3f4-0.avif" alt="Output of the previous code cell" />

## Step 2: Optimize problem for quantum hardware execution
In this step, we proceed with optimization using the AI synthesis passes.

For the AI synthesis passes, the `PassManager` requires only the coupling map of the backend. However, it is important to note that not all coupling maps are compatible; only those that the `AIPermutationSynthesis` pass has been trained on will work. Currently, the `AIPermutationSynthesis` pass supports blocks of sizes 65, 33, and 27 qubits. For this example we use a 27-qubit QPU.

For comparison, we will evaluate the performance of AI synthesis against generic permutation synthesis methods in Qiskit, including:

- `synth_permutation_depth_lnn_kms`: This method synthesizes a permutation circuit for a linear nearest-neighbor (LNN) architecture using the Kutin, Moulton, and Smithline (KMS) algorithm. It guarantees a circuit with a depth of at most $ n $ and a size of at most $ n(n-1)/2 $, where both depth and size are measured in terms of SWAP gates.

- `synth_permutation_basic`: This is a straightforward implementation that synthesizes permutation circuits without imposing constraints on connectivity or optimization for specific architectures. It serves as a baseline for comparing performance with more advanced methods.

Each of these methods represents a distinct approach to synthesizing permutation networks, providing a comprehensive benchmark against the AI-powered methods.

For more details about synthesis methods in Qiskit, refer to the [Qiskit API documentation](/docs/api/qiskit/synthesis).

Define the coupling map representing the 27-qubit QPU.

In [11]:
coupling_map = [
    [1, 0],
    [2, 1],
    [3, 2],
    [3, 5],
    [4, 1],
    [6, 7],
    [7, 4],
    [7, 10],
    [8, 5],
    [8, 9],
    [8, 11],
    [11, 14],
    [12, 10],
    [12, 13],
    [12, 15],
    [13, 14],
    [16, 14],
    [17, 18],
    [18, 15],
    [18, 21],
    [19, 16],
    [19, 22],
    [20, 19],
    [21, 23],
    [23, 24],
    [25, 22],
    [25, 24],
    [26, 25],
]
CouplingMap(coupling_map).draw()

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/84dff2c2-a496-4828-bb8e-08d373816a36-0.avif" alt="Output of the previous code cell" />

Durchschnittlichi prozäntuali Reduzierig für jedi Metrik. Positivi sind Verbesserige, negativi sind Verschlechterige.

In [12]:
results = []
pm_no_ai_synth = generate_preset_pass_manager(
    coupling_map=cm,
    optimization_level=1,  # set to 1 since we are using the synthesis methods
)

# Transpile and analyze all circuits
for i, (qc_name, qc) in enumerate(circuits.items(), start=1):
    pattern = patterns[i - 1]  # Get the corresponding pattern

    qc_depth_lnn_kms = synth_permutation_depth_lnn_kms(pattern)
    qc_basic = synth_permutation_basic(pattern)

    # AI synthesis
    results.append(
        synth_transpile_with_metrics(
            qc.decompose(reps=3),
            pm_ai,
            qc_name,
            "AI",
        )
    )

    # Depth-LNN-KMS Method
    results.append(
        synth_transpile_with_metrics(
            qc_depth_lnn_kms.decompose(reps=3),
            pm_no_ai_synth,
            qc_name,
            "Depth-LNN-KMS",
        )
    )

    # Basic Method
    results.append(
        synth_transpile_with_metrics(
            qc_basic.decompose(reps=3),
            pm_no_ai_synth,
            qc_name,
            "Basic",
        )
    )


results_df = pd.DataFrame(results)

Record the metrics (depth, gate count, time) for each circuit after transpilation.

In [13]:
# Calculate averages for each metric
average_metrics = results_df.groupby("Method")[
    ["Depth (2Q)", "Gates", "Time (s)"]
].mean()
average_metrics = average_metrics.round(3)  # Round to two decimal places
print("\n=== Average Metrics ===")
print(average_metrics)

# Identify the best non-AI method based on least average depth
non_ai_methods = [
    method for method in results_df["Method"].unique() if method != "AI"
]
best_non_ai_method = average_metrics.loc[non_ai_methods][
    "Depth (2Q)"
].idxmin()
print(
    f"\nBest Non-AI Method (based on least average depth): {best_non_ai_method}"
)

# Compare AI to the best non-AI method
ai_metrics = average_metrics.loc["AI"]
best_non_ai_metrics = average_metrics.loc[best_non_ai_method]

comparison = {
    "Metric": ["Depth (2Q)", "Gates", "Time (s)"],
    "AI": [
        ai_metrics["Depth (2Q)"],
        ai_metrics["Gates"],
        ai_metrics["Time (s)"],
    ],
    best_non_ai_method: [
        best_non_ai_metrics["Depth (2Q)"],
        best_non_ai_metrics["Gates"],
        best_non_ai_metrics["Time (s)"],
    ],
    "Improvement (AI vs Best Non-AI)": [
        ai_metrics["Depth (2Q)"] - best_non_ai_metrics["Depth (2Q)"],
        ai_metrics["Gates"] - best_non_ai_metrics["Gates"],
        ai_metrics["Time (s)"] - best_non_ai_metrics["Time (s)"],
    ],
}

comparison_df = pd.DataFrame(comparison)
print("\n=== Comparison of AI vs Best Non-AI Method ===")
comparison_df


=== Average Metrics ===
               Depth (2Q)  Gates  Time (s)
Method                                    
AI                   23.9   82.8     0.248
Basic                29.8   91.0     0.012
Depth-LNN-KMS        70.8  531.6     0.017

Best Non-AI Method (based on least average depth): Basic

=== Comparison of AI vs Best Non-AI Method ===


,Metric,AI,Basic,Improvement (AI vs Best Non-AI)
0,Depth (2Q),23.900,29.800,-5.900
1,Gates,82.800,91.000,-8.200
2,Time (s),0.248,0.012,0.236


The results demonstrate that the AI transpiler outperforms all other Qiskit synthesis methods for this set of random permutation circuits. Key findings include:

1. Depth: The AI transpiler achieves the lowest average depth, indicating superior optimization of circuit layouts.
2. Gate count: It significantly reduces the number of gates compared to other methods, improving execution fidelity and efficiency.
3. Transpilation time: All methods run very quickly at this scale, making them practical for use. However, the AI transpiler does has a notable runtime increase compared to traditional methods due to the complexity of the AI models used.

These results establish the AI transpiler as the most effective approach for this benchmark, particularly for depth and gate count optimization.

Plot the results to compare the performance of the AI synthesis passes against the generic synthesis methods.

In [14]:
methods = results_df["Method"].unique()

fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Pivot the DataFrame and reorder columns to ensure AI is first
pivot_depth = results_df.pivot(
    index="Pattern", columns="Method", values="Depth (2Q)"
)[["AI", "Depth-LNN-KMS", "Basic"]]
pivot_gates = results_df.pivot(
    index="Pattern", columns="Method", values="Gates"
)[["AI", "Depth-LNN-KMS", "Basic"]]
pivot_time = results_df.pivot(
    index="Pattern", columns="Method", values="Time (s)"
)[["AI", "Depth-LNN-KMS", "Basic"]]

pivot_depth.plot(kind="bar", ax=axs[0], legend=False)
axs[0].set_title("Circuit Depth Comparison")
axs[0].set_ylabel("Depth")
axs[0].set_xlabel("Pattern")
axs[0].tick_params(axis="x", rotation=45)
pivot_gates.plot(kind="bar", ax=axs[1], legend=False)
axs[1].set_title("2Q Gate Count Comparison")
axs[1].set_ylabel("Number of 2Q Gates")
axs[1].set_xlabel("Pattern")
axs[1].tick_params(axis="x", rotation=45)
pivot_time.plot(
    kind="bar", ax=axs[2], legend=True, title="Legend"
)  # Show legend on the last plot
axs[2].set_title("Time Comparison")
axs[2].set_ylabel("Time (seconds)")
axs[2].set_xlabel("Pattern")
axs[2].tick_params(axis="x", rotation=45)
fig.suptitle(
    "Benchmarking AI Synthesis Methods vs Non-AI Synthesis Methods For Random Permutations Circuits",
    fontsize=16,
    y=1,
)

plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/a326f268-0115-442c-8563-968676b66670-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/79b8d5d9-0f9d-42ca-9583-8bec17430014-0.avif)

D'Leistig vom KI-Transpiler variiert je no Art vo de z'optimierende Schaltig erheblich. I einzelne Fäll erreicht er bemarkenswärti Reduzierige vo de Schaltigstieffi und Gate-Aazahl im Vergliich zum Standard-Transpiler. Dää Verbesserige gönd aber oft mit ere erhebliche Erhöhig vo de Loufziit iher.

Für bestimmti Schaltigstype chan de KI-Transpiler e bizzli besseri Ergebnis bezüglich de Schaltigstieffi erziehle, aber ou zu ere Erhöhig vo de Gate-Aazahl und ere erhebliche Loufziitschtroof füehre. Dää Beobachtige lege noh, dass d'Vortel vom KI-Transpiler nöd bi allne Schaltigstype einheitlich sind. Stattdesse hängt siini Effektivität vo de spezifische Eigeschafte vo de Schaltig ab, was en für gwüssi Aawändigsfäll besser geignet macht als für anderi.

## Wänn sölled Benutzer KI-gstützti Transpilation wähle?

De KI-gstützt Transpiler i Qiskit glänzt i Szenarios, wo traditionelli Transpilationsmethode Schwiirigkeite hend, bsunders bi gross aaleite und komplexe Quanteschaltige. Für Schaltige mit Hunderte vo Qubits oder die, wo uf Hardware mit komplizierte Coupling-Maps abziehle, bietet de KI-Transpiler überlëgeni Optimierig bezüglich Schaltigstieffi, Gate-Aazahl und Loufziiteffizïänz. I Benchmarking-Teschts het er traditionelli Methode konsischtent übertroffe und düütlich flacheri Schaltige glieferet und Gate-Aazahle reduziert, was für d'Verbesserig vo de Leistig und d'Minderig vo Ruusche uf echter Quantehardware entscheidend isch.

Benutzer sölled KI-gstützti Transpilation i Betracht zieh, wänn sie mit:
- Grosse Schaltige schaffe, wo traditionelli Methode de Maassstab nöd effizient hanthabe chöi.
- Komplexe Hardware-Topologie, wo Gerät-konnektivität und Routing-Usefordrige uftrete.
- Leistigssensitivi Aawändige, wo d'Reduzierig vo de Schaltigstieffi und d'Verbesserig vo de Fidelität vo gröschter Bedütig sind.

# Teil III. Erkundig vo de KI-gstützte Permutationsnetzwärk-Synthese

Permutationsnetzwärk sind grundlegend im Quantecomputing, bsunders für Systeme, wo dur iigschränkti Topologie beschränkt sind. Dää Netzwärk erliichtere Langschtreckèinteraktione, indem sie Qubits dynamisch tausche, zum All-to-All-Konnektivität uf Hardware mit begränzter Konnektivität nohzbauè. Sölchi Transformatione sind für d'Implementierig komplexer Quantealgorithme uf churzfrischtigi Gerät unerlässlich, wo Interaktione oft über nöchschti Nohbere uusegönd.

I däm Abschnitt hebe mir d'Synthese vo Permutationsnetzwärk als überzüügendi Aawändig für de KI-gstützt Transpiler i Qiskit füre. Bsunders nutzt de `AIPermutationSynthesis`-Pass KI-gstüürti Optimierig, zum effizïänti Schaltige für Qubit-Permutationsufgabe z'generiere. Im Gëgesatz dezue hend generischi Synthese-Aasätz oft Schwiirigkeite, Gate-Aazahl und Schaltigstieffi uuszbalanziere, bsunders i Szenarios mit dichtte Qubit-Interaktione oder bim Versuech, volli Konnektivität z'erreiche.

Mir wärde es Qiskit-Musterbiischpil durgöh, wo d'Synthese vo eme Permutationsnetzwärk zeigt, zum All-to-All-Konnektivität für e Satz vo Qubits z'erreiche. Mir wärde d'Leistig vo `AIPermutationSynthesis` mit de Standard-Synthesemethode i Qiskit vergliche. Das Biischpil wird zeige, wie de KI-Transpiler für geringeri Schaltigstieffi und Gate-Aazahl optimiert und siini Vorteil i praktische Quante-Workflows fürevöhebe. Zum de KI-Synthese-Pass z'aktiviere, verwände mir d'Funktion `generate_ai_pass_manager()` mit em Parameter `include_ai_synthesis` uf `True` gsetzt.

## Schritt 1: Klassischi Iigabe uf es Quanteproblem abilde

Zum es klassischs Permutationsproblem uf eme Quantecomputer darzustellè, fange mir mit de Definition vo de Struktur vo de Quanteschaltige aa. Für das Biischpil:

1. Quanteschaltig-Initialisierig:
   Mir weise 27 Qubits zue, zum zum Backend z'passe, wo mir verwände wärde, wo 27 Qubits het.

2. Permutatione aawände:
   Mir generiere zehn zufälligi Permutationsmuster (`pattern_1` bis `pattern_10`) under Verwendig vo eme feschte Seed für Reproduzierbarkeit. Jedes Permutationsmuster wird uf e separati Quanteschaltig aagwändet (`qc_1` bis `qc_10`).

3. Schaltigszerlëgig:
   Jedi Permutationsoperation wird i nativi Gate-Sets zerleit, wo mit de Ziil-Quantehardware kompatibel sind. Mir analysiere d'Tieffi und d'Aazahl vo de Zwei-Qubit-Gates (nöd-lokali Gates) für jedi zerleiti Schaltig.

D'Ergebnis gäbe Iblick i d'Komplexität vo de Darstellig klassischi Permutationsproblem uf eme Quantegerät und demonstriere d'Ressourceaaforderige für verschideni Permutationsmuster.